In [1]:
import numpy as np
import pandas as pd

In [2]:
import tensorflow as tf
import tensorflow.keras as k
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Input, Conv2D, MaxPool2D, Flatten, Reshape

In [3]:
import silence_tensorflow

In [4]:
from keras_bed_sequence import BedSequence
from keras_mixed_sequence import MixedSequence

# Data

In [5]:
df = pd.read_csv("./mendelian_snv.csv.gz")

In [6]:
chromosomes = df.chrom.unique()

In [7]:
train_bed = df.head(int(1e5))

In [8]:
import numpy as np
import pandas as pd
from keras_bed_sequence import BedSequence
from keras_mixed_sequence import MixedSequence
from ucsc_genomes_downloader.utils import wiggle_bed_regions
from holdouts_generator import holdouts_generator, balanced_random_holdouts

# Split
generator = holdouts_generator(
    train_bed, train_bed.labels,
    holdouts=balanced_random_holdouts(
        [0.3],
        [1]
    )
)

training, testing = next(generator())[0]

In [10]:
def create_sequence(bed, y, assembly="hg19", batchsize=128):
    X = BedSequence(
        assembly=assembly,
        bed=bed,
        batch_size=batchsize
    )
    return MixedSequence(X, y, batchsize)

In [11]:
def wiggle(training):
    x, y = training
    positives = x[y==1]
    x = pd.concat([x, wiggle_bed_regions(positives, 150, 10, seed=42)], axis=0)
    y = x.labels.values
    print(x.shape, y.shape)
    return create_sequence(x, y)

In [12]:
train = wiggle(training)

(72509, 5) (72509,)


/home/zom/anaconda3/lib/python3.7/site-packages/ucsc_genomes_downloader/utils/wiggle_bed_regions.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bed.chromStart[bed.chromStart < 0] = 0


In [13]:
test = create_sequence(*testing)

# Model

In [14]:
from mendelian_snv_prediction import get_model

In [15]:
model = get_model()
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 500, 4)]          0         
_________________________________________________________________
reshape (Reshape)            (None, 500, 4, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 500, 4, 128)       4736      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 166, 4, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 166, 4, 64)        98368     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 83, 4, 64)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 83, 4, 64)         12352 

# Test

In [16]:
history = model.fit_generator(
    generator=train,
    steps_per_epoch=train.steps_per_epoch,
    validation_data=test,
    validation_steps=test.steps_per_epoch,
    epochs=100,
    verbose=1,
    use_multiprocessing=False,
    shuffle=True
).history

Train for 567 steps, validate for 235 steps
Epoch 1/100
567/567 [==============================] - 293s 517ms/step - loss: 42.9048 - val_loss: 0.3010
Epoch 2/100
567/567 [==============================] - 302s 532ms/step - loss: 42.9048 - val_loss: 0.3010
Epoch 3/100
567/567 [==============================] - 329s 580ms/step - loss: 42.9048 - val_loss: 0.3010
Epoch 4/100
 15/567 [..............................] - ETA: 5:08 - loss: 88.7228 

KeyboardInterrupt: 